In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

import random

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
%%time
from transplant.tools.learningset import Learningset

learningset = Learningset()

X_train, X_test , y_train, y_test , X_col = learningset.get_data_merged_dynamic("One_Hot")
X_train_not_use, X_test_not_use , y_train_cls, y_test_cls , X_col_not_use = learningset.get_data_merged_dynamic()

Wall time: 20.8 s


In [8]:
n_inputs = X_train.shape[1]  
n_hidden1 = 35
n_hidden2 = 10
n_outputs = 2

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.shape[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.1

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()


In [9]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [10]:
%%time
n_epochs = 1000
batch_size = 150

l_accu=[]
l_batch=[]
l_epoch=[]

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train_cls, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_test, y: y_test_cls})
        if epoch%100==0 :
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
            l_epoch+=[epoch]
            l_batch+=[acc_batch]
            l_accu+=[acc_valid]
            
            
plt.plot(l_epoch,l_batch)
plt.plot(l_epoch,l_accu)

TypeError: float() argument must be a string or a number, not 'Timestamp'